In [0]:
#pip install spark-nlp==3.0.0

Python interpreter will be restarted.
Collecting spark-nlp==3.0.0
 Downloading spark_nlp-3.0.0-py2.py3-none-any.whl (140 kB)
Installing collected packages: spark-nlp
Successfully installed spark-nlp-3.0.0
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-8892ccae-12cb-4260-886f-093279666de0/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.embeddings import *
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline

In [0]:
spark=sparknlp.start()

In [0]:
print(spark.version)
print(sparknlp.version())

3.1.0
3.0.0

In [0]:
data=spark.createDataFrame([['Peter is a godo person living in Germny. Paula is a good person. She lives in London.']]).toDF('text')
data.show(truncate=False)

+-------------------------------------------------------------------------------------+
text |
+-------------------------------------------------------------------------------------+
Peter is a godo person living in Germny. Paula is a good person. She lives in London.|
+-------------------------------------------------------------------------------------+

In [0]:
document=DocumentAssembler().setInputCol("text").setOutputCol("document").setCleanupMode("shrink")

In [0]:
print(document)

DocumentAssembler_c50d1c20c03f

In [0]:
sentence=SentenceDetector().setInputCols('document').setOutputCol('sentence')

In [0]:
print(sentence)

SentenceDetector_ccc095a55383

In [0]:
sentence.setExplodeSentences(True)

Out[28]: SentenceDetector_ccc095a55383

In [0]:
tokenizer=Tokenizer().setInputCols('sentence').setOutputCol('tokens')
print(tokenizer)

Tokenizer_4c92afb96522

In [0]:
checker=NorvigSweetingModel.pretrained().setInputCols(['tokens']).setOutputCol('checked')

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[ | ][OK!]

In [0]:
embeddings=WordEmbeddingsModel.pretrained().setInputCols(['sentence','checked']).setOutputCol('embeddings')

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][OK!]

In [0]:
ner=NerDLModel.pretrained().setInputCols(['sentence','checked','embeddings']).setOutputCol('ner')

ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[ | ][OK!]

In [0]:
converter=NerConverter().setInputCols(['sentence','checked','ner']).setOutputCol('chunk')

In [0]:
pipeline=Pipeline().setStages([document,sentence,tokenizer,checker,embeddings,ner,converter])

In [0]:
model=pipeline.fit(data)

In [0]:
result=model.transform(data)

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| tokens| checked| embeddings| ner| chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Peter is a godo p...|[{document, 0, 84...|[{document, 0, 39...|[{token, 0, 4, Pe...|[{token, 0, 4, Pe...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 4, Pe...|
Peter is a godo p...|[{document, 0, 84...|[{document, 41, 6...|[{token, 41, 45, ...|[{token, 41, 45, ...|[{word_embeddings...|[{named_entity, 4...|[{chunk, 41, 45, ...|
Peter is a godo p...|[{document, 0, 84...|[{document, 65, 8...|[{token, 65, 67, ...|[{token, 65, 67, ...|[{word_embeddings...|[{named_entity, 6...|[{chunk, 78, 83, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
result.select('sentence.result').show(truncate=False)

+------------------------------------------+
result |
+------------------------------------------+
[Peter is a godo person living in Germny.]|
[Paula is a good person.] |
[She lives in London.] |
+------------------------------------------+

In [0]:
result.select('checked.result').show(truncate=False)

+---------------------------------------------------+
result |
+---------------------------------------------------+
[Peter, is, a, god, person, living, in, Germany, .]|
[Paula, is, a, good, person, .] |
[She, lives, in, London, .] |
+---------------------------------------------------+

In [0]:
result.select('ner.result').show(truncate=False)

+-----------------------------------+
result |
+-----------------------------------+
[B-PER, O, O, O, O, O, O, B-LOC, O]|
[B-PER, O, O, O, O, O] |
[O, O, O, B-LOC, O] |
+-----------------------------------+

In [0]:
result.select('ner.begin','ner.end').show(truncate=False)

+---------------------------------+---------------------------------+
begin |end |
+---------------------------------+---------------------------------+
[0, 6, 9, 11, 16, 23, 30, 33, 39]|[4, 7, 9, 14, 21, 28, 31, 38, 39]|
[41, 47, 50, 52, 57, 63] |[45, 48, 50, 55, 62, 63] |
[65, 69, 75, 78, 84] |[67, 73, 76, 83, 84] |
+---------------------------------+---------------------------------+

In [0]:
result.select('chunk.result','chunk.begin','chunk.end').show(truncate=False)

+---------------+-------+-------+
result |begin |end |
+---------------+-------+-------+
[Peter, Germny]|[0, 33]|[4, 38]|
[Paula] |[41] |[45] |
[London] |[78] |[83] |
+---------------+-------+-------+

In [0]:
light=LightPipeline(model)

In [0]:
light.annotate('Peter is a godo person living in Germny. Paula is a good person. She lives in London.')

Out[44]: {'chunk': ['Peter', 'Germny', 'Paula', 'London'],
 'checked': ['Peter',
 'is',
 'a',
 'god',
 'person',
 'living',
 'in',
 'Germany',
 '.',
 'Paula',
 'is',
 'a',
 'good',
 'person',
 '.',
 'She',
 'lives',
 'in',
 'London',
 '.'],
 'document': ['Peter is a godo person living in Germny. Paula is a good person. She lives in London.'],
 'ner': ['B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O'],
 'tokens': ['Peter',
 'is',
 'a',
 'godo',
 'person',
 'living',
 'in',
 'Germny',
 '.',
 'Paula',
 'is',
 'a',
 'good',
 'person',
 '.',
 'She',
 'lives',
 'in',
 'London',
 '.'],
 'embeddings': ['Peter',
 'is',
 'a',
 'god',
 'person',
 'living',
 'in',
 'Germany',
 '.',
 'Paula',
 'is',
 'a',
 'good',
 'person',
 '.',
 'She',
 'lives',
 'in',
 'London',
 '.'],
 'sentence': ['Peter is a godo person living in Germny.',
 'Paula is a good person.',
 'She lives in London.']}

In [0]:
light.annotate('let us explore some more models in Spark NLP. We are happy to learn about Spark NLP.')

Out[45]: {'chunk': ['Spark NLP', 'Spark NLP'],
 'checked': ['let',
 'us',
 'explore',
 'some',
 'more',
 'models',
 'in',
 'Spark',
 'NLP',
 '.',
 'We',
 'are',
 'happy',
 'to',
 'learn',
 'about',
 'Spark',
 'NLP',
 '.'],
 'document': ['let us explore some more models in Spark NLP. We are happy to learn about Spark NLP.'],
 'ner': ['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O'],
 'tokens': ['let',
 'us',
 'explore',
 'some',
 'more',
 'models',
 'in',
 'Spark',
 'NLP',
 '.',
 'We',
 'are',
 'happy',
 'to',
 'learn',
 'about',
 'Spark',
 'NLP',
 '.'],
 'embeddings': ['let',
 'us',
 'explore',
 'some',
 'more',
 'models',
 'in',
 'Spark',
 'NLP',
 '.',
 'We',
 'are',
 'happy',
 'to',
 'learn',
 'about',
 'Spark',
 'NLP',
 '.'],
 'sentence': ['let us explore some more models in Spark NLP.',
 'We are happy to learn about Spark NLP.']}